# Multiclass Land Cover Classification using Random Forest 🌲

This notebook performs **8-class land cover classification** using Random Forest with Sentinel-2 satellite imagery and Google Earth Engine.

**Classes:**
- 0: Deep Water
- 1: Wetland/Shallow Water
- 2: Dense Forest
- 3: Mixed Forest
- 4: Grassland
- 5: Buildings
- 6: Roads
- 7: Bare Soil

### 1. Setup & Initialization

In [ ]:
import os
from dotenv import load_dotenv
import ee
import geemap

# Load environment variables
load_dotenv()

# Initialize Earth Engine
ee_project = os.getenv('EE_PROJECT_ID')
if not ee_project:
    raise ValueError("EE_PROJECT_ID not set in .env file")

ee.Initialize(project=ee_project)

# Random seed for reproducibility
SEED = 42

# Class configuration
NUM_CLASSES = 8
CLASS_NAMES = ['Deep Water', 'Wetland', 'Dense Forest', 'Mixed Forest', 
               'Grassland', 'Buildings', 'Roads', 'Bare Soil']
CLASS_PALETTE = ['0000FF', '00BFFF', '006400', '228B22', 
                 '90EE90', 'FF0000', '808080', 'D2B48C']

### 2. Define Study Area

In [ ]:
# Campus boundary (replace with your study area)
campus_geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [80.01710357666015, 23.173962177472703],
            [80.03259601593017, 23.165361215115187],
            [80.03654422760009, 23.172502420044232],
            [80.026802444458, 23.181694681000845],
            [80.01542987823485, 23.176960548201308],
        ]
    ]
}
study_area = ee.Geometry(campus_geojson)

### 3. Load Sentinel-2 Imagery

In [ ]:
def mask_s2_clouds(image):
    """Mask clouds in Sentinel-2 imagery."""
    qa = image.select('QA60')
    cloud = 1 << 10
    cirrus = 1 << 11
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(cirrus).eq(0))
    return image.updateMask(mask).divide(10000)

# Load and preprocess Sentinel-2 data
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate('2025-11-15', '2026-01-15')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
           .map(mask_s2_clouds))

image = dataset.median().clip(study_area)

In [ ]:
# Visualize RGB composite
Map = geemap.Map()
Map.centerObject(study_area, 16)
Map.addLayer(image, {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3,
}, 'RGB Composite')
Map

### 4. Calculate Spectral Indices

In [ ]:
# NDVI - Vegetation Index
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# NDWI - Water Index
ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')

# NDBI - Built-up Index
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Add indices to image
image = image.addBands([ndvi, ndwi, ndbi])

# Feature bands for classification
bands = ['B4', 'B8', 'B3', 'B11', 'NDVI', 'NDWI', 'NDBI']

### 5. Load Training Data

**Note:** Ensure all 8 training point assets exist in your Earth Engine Assets with a `label` property (0-7).

In [ ]:
# Define training point asset paths
# Replace 'cosypix' with your GEE username
training_assets = {
    0: 'users/cosypix/deep_water_points',
    1: 'users/cosypix/wetland_points',
    2: 'users/cosypix/dense_forest_points',
    3: 'users/cosypix/mixed_forest_points',
    4: 'users/cosypix/grassland_points',
    5: 'users/cosypix/buildings_points',
    6: 'users/cosypix/roads_points',
    7: 'users/cosypix/bare_soil_points',
}

try:
    # Load and merge all training points
    training_points = None
    for label, asset_path in training_assets.items():
        points = ee.FeatureCollection(asset_path)
        if training_points is None:
            training_points = points
        else:
            training_points = training_points.merge(points)
    
    print(f"Loaded training points successfully")
    print(f"Total training points: {training_points.size().getInfo()}")
    
except Exception as e:
    print("Error loading training assets. Make sure all 8 assets exist.")
    print(e)

### 6. Sample Training Data

In [ ]:
if 'training_points' in locals():
    # Sample spectral values at training points
    training = image.select(bands).sampleRegions(
        collection=training_points,
        properties=['label'],
        scale=10
    )
    
    # Remove null values
    training = training.filter(ee.Filter.notNull(bands + ['label']))
    print(f"Training samples: {training.size().getInfo()}")

### 7. Train/Test Split

In [ ]:
if 'training' in locals():
    # Add random column for splitting
    training = training.randomColumn('random', SEED)
    
    # 70% train, 30% test
    train_set = training.filter(ee.Filter.lt('random', 0.7))
    test_set = training.filter(ee.Filter.gte('random', 0.7))
    
    print(f"Training samples: {train_set.size().getInfo()}")
    print(f"Test samples: {test_set.size().getInfo()}")

### 8. Train Random Forest Classifier

In [ ]:
if 'train_set' in locals():
    # Train Random Forest classifier with 150 trees
    classifier = ee.Classifier.smileRandomForest(150).train(
        features=train_set,
        classProperty='label',
        inputProperties=bands
    )
    print("Random Forest classifier trained successfully!")

### 9. Accuracy Assessment

In [ ]:
if 'test_set' in locals() and 'classifier' in locals():
    # Classify test set
    validated = test_set.classify(classifier)
    
    # Compute confusion matrix
    confusion_matrix = validated.errorMatrix('label', 'classification')
    
    print("Confusion Matrix:")
    print(confusion_matrix.getInfo())
    print(f"\nOverall Accuracy: {confusion_matrix.accuracy().getInfo():.4f}")
    print(f"Kappa Coefficient: {confusion_matrix.kappa().getInfo():.4f}")
    
    # Per-class metrics
    print("\nProducer's Accuracy (per class):")
    producers = confusion_matrix.producersAccuracy().getInfo()
    for i, acc in enumerate(producers):
        if i < len(CLASS_NAMES):
            print(f"  {CLASS_NAMES[i]}: {acc[0]:.4f}")

### 10. Feature Importance

In [ ]:
if 'classifier' in locals():
    # Get feature importance from Random Forest
    importance = classifier.explain().getInfo()
    print("Feature Importance:")
    if 'importance' in importance:
        for band, score in importance['importance'].items():
            print(f"  {band}: {score:.4f}")

### 11. Classify Study Area

In [ ]:
if 'classifier' in locals():
    # Classify the entire study area
    classified = image.select(bands).classify(classifier)
    
    # Apply smoothing (focal mode)
    smooth_classification = classified.focal_mode(1)
    print("Classification complete!")

### 12. Visualize Results

In [ ]:
Map = geemap.Map()
Map.centerObject(study_area, 16)

# Add RGB composite
Map.addLayer(image, {
    'bands': ['B4', 'B3', 'B2'], 
    'min': 0,
    'max': 0.3
}, 'RGB Composite')

# Add classification result
Map.addLayer(smooth_classification, {
    'min': 0,
    'max': NUM_CLASSES - 1,
    'palette': CLASS_PALETTE
}, 'Land Cover Classification')

# Add legend
legend_dict = dict(zip(CLASS_NAMES, ['#' + c for c in CLASS_PALETTE]))
Map.add_legend(title='Land Cover Classes', legend_dict=legend_dict)

Map